In [50]:
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import time
import numpy as np
import os

In [26]:
def ssim_compare(img1_path, img2_path, dim=(112, 112)):
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        raise ValueError("One or both image paths are invalid or unreadable.")

    if img1.shape != dim:
        img1 = cv2.resize(img1, dim, interpolation=cv2.INTER_AREA)
    if img2.shape != dim:
        img2 = cv2.resize(img2, dim, interpolation=cv2.INTER_AREA)

    return ssim(img1, img2)

def ssim_compare_frames(frame1, frame2, dim=(112, 112)):
    if frame1.shape != dim:
        img1 = cv2.resize(frame1, dim, interpolation=cv2.INTER_AREA)
    if frame2.shape != dim:
        img2 = cv2.resize(frame2, dim, interpolation=cv2.INTER_AREA)

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return ssim(gray1, gray2)


In [21]:
url_image1 = "./CUT3R/examples/005/frame_0.png"
url_image2 = "./CUT3R/examples/005/frame_1.png"

start = time.perf_counter()
ssim_score = ssim_compare(url_image1, url_image2)
end = time.perf_counter()

print(f"Execution time: {end - start:.6f} seconds")
print(f"SSIM Score: {ssim_score}")

Execution time: 0.020321 seconds
SSIM Score: 0.9971229066536855


In [56]:
input_vid = './CUT3R/examples/vid_test.mp4'
output_dir = "frame_parse"
dim=(112,112)

os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(input_vid)

frame_idx = 0
last_saved_frame = None
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if last_saved_frame is not None:
        start = time.perf_counter()
        score = ssim_compare_frames(last_saved_frame, frame)
        end = time.perf_counter()
        # print(f"Frame {frame_idx} SSIM score with previous frame: {score:.6f} (computed in {end - start:.6f} seconds)")

        if score < .5:
            # print("Less than 50% similariy, save")

            full_path = os.path.join(output_dir, f"frame_{frame_idx}.png")
            cv2.imwrite(full_path, frame)

            last_saved_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

    else:
        full_path = os.path.join(output_dir, f"frame_{frame_idx}.png")
        cv2.imwrite(full_path, frame)
        last_saved_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    
    prev_frame = frame
    frame_idx += 1


cap.release()
cv2.destroyAllWindows()